## Customer Transaction Prediction

이 프로젝트의 목표는 고객들의 히스토리 데이터를 분석하여, 고객의 거래 된 금액에 관계없이, 향후 어떤 고객이 특정 거래를 할 것인지 파악할 수 있도록 하는것이 목표입니다. `target`이 0이면 거래를 할 수 없다는 것을 의미하고, 1이면 거래를 할 수 있다는 것을 의미합니다.

 - Target의 분포를 살펴보면 0과 1의 비율이 맞지 않습니다. 즉 imbalance 데이터 입니다. 이런 데이터를 다루는 기법중의 하나는 두 데이터의 크기를 1:1로 맞추는 것입니다. 두 카테고리의 데이터 수가 맞도록 샘플링 하세요.
 - 우리의 목표는 주어진 데이터를 활용하여 즉 `var_0` ~ `var_99`를 이용하여 `target`을 예측하는 것입니다. 분류기중에서는 우리가 배웠던 naive bayes를 활용합니다.

In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np

/Users/sanghyub/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
pd.options.display.max_columns = 300

In [4]:
data = pd.read_csv('inputs/train.csv')

In [5]:
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,2.1743,-3.4132,9.4763,13.3102,26.5376,1.4403,14.7100,6.0454,9.5426,17.1554,14.1104,24.3627,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.5752,-13.4221,7.2739,16.0094,9.7268,0.8897,0.7754,4.2218,12.0039,13.8571,-0.7338,-1.9245,15.4462,12.8287,0.3587,9.6508,6.5674,5.1726,3.1345,29.4547,31.4045,2.8279,15.6599,8.3307,-5.6011,19.0614,11.2663,8.6989,8.3694,11.5659,-16.4727,4.0288,17.9244,18.5177,10.7800,9.0056,16.6964,10.4838,1.6573,12.1749,-13.1324,17.6054,11.5423,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,10.8160,3.6783,-11.1147,1.8730,9.8775,11.7842,1.2444,-47.3797,7.3718,0.1948,34.4014,25.7037,11.8343,13.2256,-4.1083,6.6885,-8.0946,18.5995,19.3219,7.0118,1.9210,8.8682,8.0109,-7.2417,1.7944,-1.3147,8.1042,1.5365,5.4007,7.9344,5.0220,2.2302,40.5632,0.5134,3.1701,20.1068,7.7841,7.0529,3.2709,23.4822,5.5075,13.7814,2.5462,18.1782,0.3683,-4.8210,-5.4850,13.7867,-13.5901,11.0993,7.9022,12.2301,0.4768,6.8852,8.0905,10.9631,11.7569,-1.2722,24.7876,26.6881,1.8944,0.6939,-13.6950,8.4068,35.4734,1.7093,15.1866,2.6227,7.3412,32.0888,13.9550,13.0858,6.6203,7.1051,5.3523,8.5426,3.6159,4.1569,3.0454,7.8522,-11.5100,7.5109,31.5899,9.5018,8.2736,10.1633,0.1225,12.5942,14.5697

In [6]:
data.shape

(200000, 202)

In [7]:
data.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2000

In [8]:
sum(data.isnull().sum())

0

In [9]:
zero_target = data[data.target == 0]

In [10]:
one_target = data[data.target == 1]

In [11]:
print(zero_target.shape, one_target.shape)

(179902, 202) (20098, 202)


target 0인 것에서 stratified sampling?

In [27]:
zero_sampled = zero_target.sample(n=20098, replace=False, random_state=42)

In [30]:
zero_sampled.shape

(20098, 202)

In [31]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

In [32]:
sampled_data = pd.concat([zero_sampled, one_target])

In [33]:
del sampled_data['ID_code']

In [34]:
sampled_data.shape

(40196, 201)

In [35]:
sampled_data.head() #shuffle 필요

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
120837,0,7.1545,2.8296,12.8612,7.6127,10.6555,-0.0555,4.4894,24.4828,-4.7375,5.4653,5.4987,4.0326,14.1648,14.8932,5.3732,14.3354,12.0535,2.0397,9.6205,2.2053,5.0263,12.6342,4.9617,2.5920,15.1312,13.2126,-5.5624,-0.6735,6.1594,2.7483,-7.6748,8.2910,0.2935,11.2861,11.4812,12.7597,5.2913,3.1557,6.2991,-1.0226,0.8704,19.1938,10.9609,11.5695,2.7934,-20.2595,13.1295,3.8214,24.2765,18.1106,11.4546,21.7254,3.6387,5.2954,3.8620,21.1109,13.5834,7.1742,1.5678,8.8794,8.8568,-6.9873,3.0812,4.2662,7.1489,9.4119,6.1731,-3.8011,5.0241,-1.8815,10.6960,0.8259,2.3129,21.3309,-7.3532,11.7190,-5.5652,20.0806,5.4523,15.4694,9.4420,15.3357,6.7343,-2.3178,-6.6122,13.9624,6.5475,15.5880,5.8777,2.8648,-26.8661,6.7878,16.4981,10.3528,9.1651,-0.5346,12.5492,29.0375,0.4953,-3.8393,-10.8542,6.2363,27.0252,1.6286,11.3012,3.9286,10.9466,20.0673,14.3691,20.9218,4.6867,6.0891,5.3013,6.8522,3.2302,4.1673,3.8674,12.5747,-5.4918,3.0363,25.2048,7.4877,-1.9776,4.6263,6.9828,12.2172,12.5885,-0.0245,-4.9442,11.3991,13.1028,0.7388,5.7311,6.7304,6.4968,-7.4120,10.4640,2.2335,-2.7646,5.7897,7.0107,12.8615,6.2755,12.0417,8.4373,9.4224,13.1111,-13.6826,4.0329,5.3543,15.9820,15.5237,9.2258,17.8940,0.3991,0.5719,13.5903,-15.2134,28.4186,6.8459,35.4038,5.9184,3.0019,4.7777,-2.2666,18.6124,3.4983,-1.4536,4.6718,5.2365,3.2191,0.1962,20.7345,1.2320,19.8840,12.1416,-3.2583,13.4073,10.9618,0.3995,-3.0923,11.4096,11.1233,10.1327,23.4611,-13.1675,7.2833,4.4570,14.6822,2.6802,0.4694,9.5021,4.2713,-4.1652,15.8793,-1.6903,5.5892,8.2131,16.4736,-11.5577
191522,0,7.7315,-3.4498,7.3103,7.9163,9.4726,-9.8974,4.9925,19.3458,3.8757,6.3723,1.6583,5.8721,14.0540,4.9426,10.5936,14.5028,10.1665,1.3285,4.8839,17.1534,3.5761,16.8142,6.5253,3.4893,15.8507,13.9894,-14.4713,-3.2013,4.5718,3.4005,-0.5513,10.8584,2.5593,11.8538,10.2422,8.1021,1.1278,3.0426,13.2840,-0.6388,-6.6083,9.1274,11.7075,11.9605,19.2879,9.3970,15.2389,-5.5777,28.6222,15.8325,11.8358,16.1071,-1.7356,4.8701,-19.8043,7.8265,15.2319,6.9188,0.8075,10.5983,7.4423,2.5825,3.3204,-2.4480,6.7477,-3.8202,2.8669,18.4947,5.0155,2.3639,28.2316,0.6150,0.9724,18.7155,41.8846,15.0521,11.3081,23.3844,4.6177,14.5443,5.9704,14.4627,4.6835,-8.0554,-13.4601,20.9931,3.2044,16.6532,5.6375,3.6801,-10.9290,7.0566,5.7874,11.0017,10.6756,0.1918,25.6816,16.4327,1.0242,-0.3832,-22.7945,13.6288,16.3644,1.7568,11.6420,4.3775,12.6503,25.8807,14.3720,13.6630,7.0328,6.9000,3.9083,8.9766,3.8438,5.2346,2.8550,-6.4906,-16.7942,-1.2372,2.2749,10.4193,7.5756,13.6397,5.0076,12.1669,14.3396,6.0503,-2

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
data_copy = sampled_data.copy()

In [38]:
del data_copy['target']

In [39]:
data_copy.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
120837,7.1545,2.8296,12.8612,7.6127,10.6555,-0.0555,4.4894,24.4828,-4.7375,5.4653,5.4987,4.0326,14.1648,14.8932,5.3732,14.3354,12.0535,2.0397,9.6205,2.2053,5.0263,12.6342,4.9617,2.5920,15.1312,13.2126,-5.5624,-0.6735,6.1594,2.7483,-7.6748,8.2910,0.2935,11.2861,11.4812,12.7597,5.2913,3.1557,6.2991,-1.0226,0.8704,19.1938,10.9609,11.5695,2.7934,-20.2595,13.1295,3.8214,24.2765,18.1106,11.4546,21.7254,3.6387,5.2954,3.8620,21.1109,13.5834,7.1742,1.5678,8.8794,8.8568,-6.9873,3.0812,4.2662,7.1489,9.4119,6.1731,-3.8011,5.0241,-1.8815,10.6960,0.8259,2.3129,21.3309,-7.3532,11.7190,-5.5652,20.0806,5.4523,15.4694,9.4420,15.3357,6.7343,-2.3178,-6.6122,13.9624,6.5475,15.5880,5.8777,2.8648,-26.8661,6.7878,16.4981,10.3528,9.1651,-0.5346,12.5492,29.0375,0.4953,-3.8393,-10.8542,6.2363,27.0252,1.6286,11.3012,3.9286,10.9466,20.0673,14.3691,20.9218,4.6867,6.0891,5.3013,6.8522,3.2302,4.1673,3.8674,12.5747,-5.4918,3.0363,25.2048,7.4877,-1.9776,4.6263,6.9828,12.2172,12.5885,-0.0245,-4.9442,11.3991,13.1028,0.7388,5.7311,6.7304,6.4968,-7.4120,10.4640,2.2335,-2.7646,5.7897,7.0107,12.8615,6.2755,12.0417,8.4373,9.4224,13.1111,-13.6826,4.0329,5.3543,15.9820,15.5237,9.2258,17.8940,0.3991,0.5719,13.5903,-15.2134,28.4186,6.8459,35.4038,5.9184,3.0019,4.7777,-2.2666,18.6124,3.4983,-1.4536,4.6718,5.2365,3.2191,0.1962,20.7345,1.2320,19.8840,12.1416,-3.2583,13.4073,10.9618,0.3995,-3.0923,11.4096,11.1233,10.1327,23.4611,-13.1675,7.2833,4.4570,14.6822,2.6802,0.4694,9.5021,4.2713,-4.1652,15.8793,-1.6903,5.5892,8.2131,16.4736,-11.5577
191522,7.7315,-3.4498,7.3103,7.9163,9.4726,-9.8974,4.9925,19.3458,3.8757,6.3723,1.6583,5.8721,14.0540,4.9426,10.5936,14.5028,10.1665,1.3285,4.8839,17.1534,3.5761,16.8142,6.5253,3.4893,15.8507,13.9894,-14.4713,-3.2013,4.5718,3.4005,-0.5513,10.8584,2.5593,11.8538,10.2422,8.1021,1.1278,3.0426,13.2840,-0.6388,-6.6083,9.1274,11.7075,11.9605,19.2879,9.3970,15.2389,-5.5777,28.6222,15.8325,11.8358,16.1071,-1.7356,4.8701,-19.8043,7.8265,15.2319,6.9188,0.8075,10.5983,7.4423,2.5825,3.3204,-2.4480,6.7477,-3.8202,2.8669,18.4947,5.0155,2.3639,28.2316,0.6150,0.9724,18.7155,41.8846,15.0521,11.3081,23.3844,4.6177,14.5443,5.9704,14.4627,4.6835,-8.0554,-13.4601,20.9931,3.2044,16.6532,5.6375,3.6801,-10.9290,7.0566,5.7874,11.0017,10.6756,0.1918,25.6816,16.4327,1.0242,-0.3832,-22.7945,13.6288,16.3644,1.7568,11.6420,4.3775,12.6503,25.8807,14.3720,13.6630,7.0328,6.9000,3.9083,8.9766,3.8438,5.2346,2.8550,-6.4906,-16.7942,-1.2372,2.2749,10.4193,7.5756,13.6397,5.0076,12.1669,14.3396,6.0503,-2.8677,13.98

In [40]:
target = sampled_data.target

In [41]:
X_train, X_test, y_train, y_test = train_test_split(data_copy,target, test_size = 0.2, random_state=42)

In [42]:
from sklearn.preprocessing import StandardScaler

In [43]:
scaler = StandardScaler()
# 학습 셋으로만 scaling을 실시합니다.
scaler = scaler.fit(X_train)
scaled_train_x = scaler.transform(X_train)
# 학습 셋에 이용했던 scaler를 테스트 셋에 이용합니다.
scaled_test_x = scaler.transform(X_test)

In [44]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [45]:
gnb_model = gnb.fit(scaled_train_x, y_train)

In [46]:
pred = gnb_model.predict(scaled_test_x)

In [47]:
print('accuracy =', (pred==y_test).sum() / len(X_test))

accuracy = 0.8037313432835821


---
* Scaler 사용안하고 점수 확인하기

In [48]:
gnb = GaussianNB()
gnb_model = gnb.fit(X_train, y_train)

In [49]:
pred = gnb_model.predict(X_test)

In [51]:
print('accuracy :', (pred==y_test).sum()/len(X_test))

accuracy : 0.8037313432835821


똑같은 것을 확인

## 생각해볼점 (실험)
* CV를 다르게 하면 ACC가 올라갈까?
* Sclaer를 사용안하면?
* Min-Max Scaler?
* zero_sampled 데이터 샘플링할때 다른 방법을 사용하면?